# Защита информации

Вариант 9

In [4]:
# Задание #1

def decrypt(ciphertext):
    KEY = {a: b for a, b in zip('оыблеякэщмчжьфхйвёстюършпизаднугц', 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя')}

    plaintext = []
    for c in ciphertext:
        if c in KEY.keys():
            plaintext.append(KEY[c])
        else:
            plaintext.append(c)
    
    return ''.join(plaintext)

ciphertext = '''жмвм, июфм, стёяфмсн фцтяэхяч,
люем, сяейч бйейбйёйт,
еоч бяёмтн, птй ц тйэя вёяэхмч
стйг хое ёосвёяч вёяэхмр бйе!
'''

print(decrypt(ciphertext))


кипи, шуми, стремись мятежней,
гуди, седой водоворот,
дай верить, что я тоже прежний
стою над распрей прежних вод!



In [72]:
# Задание #2

def f(a, x):
    sbox = (
        (5, 15, 0, 12, 8, 4, 14, 2, 13, 11, 3, 6, 10, 1, 9, 7),
        (12, 13, 8, 14, 7, 5, 1, 6, 2, 3, 11, 0, 15, 9, 10, 4),
        (1, 10, 13, 9, 12, 0, 6, 3, 8, 15, 11, 2, 14, 4, 7, 5),
        (6, 9, 4, 2, 3, 13, 8, 14, 1, 11, 15, 7, 0, 10, 5, 12),
        (11, 0, 9, 10, 1, 2, 8, 12, 15, 3, 6, 4, 7, 5, 14, 13),
        (5, 15, 2, 1, 7, 14, 8, 6, 0, 10, 3, 9, 4, 13, 12, 11),
        (1, 0, 13, 4, 3, 2, 6, 11, 8, 5, 7, 14, 9, 10, 15, 12),
        (4, 5, 9, 11, 10, 1, 6, 13, 7, 2, 3, 0, 12, 14, 15, 8)
    )

    c = (a + x) & 0xFFFFFFFF
    seq = [(c >> (i * 4)) & 0xF for i in range(8)]
    seq = [sbox[i][seq[i]] for i in range(8)]

    out = reduce(lambda x, y: x | y, [seq[i] << (i * 4) for i in range(8)])
    return ((out >> 11) | (out << (32 - 11))) & 0xFFFFFFFF


def split(block):
    number = int.from_bytes(block, 'little')
    return number & 0xFFFFFFFF, (number >> 32) & 0xFFFFFFFF


def join(a, b):
    return ((b << 32) | a).to_bytes(8, 'little')


def flipbytes(block):
    a, b = split(block)
    return join(b, a)


def subkeys(key):
    K = [int.from_bytes(key[i * 4:i * 4 + 4], 'little') for i in range(8)]
    X = [K[i % 8] for i in range(24)] + [k for k in reversed(K)]

    return X


def encround(block, subkey):
    a, b = split(block)
    a, b = b ^ f(a, subkey), a

    return join(a, b)


def encrypt(block, key, rounds=32):
    for subkey in subkeys(key)[:rounds]:
        block = encround(block, subkey)
    return flipbytes(block)


def decrypt(block, key, rounds=32):
    for subkey in reversed(subkeys(key)[:rounds]):
        block = encround(block, subkey)
    return flipbytes(block)


plaintext = 'Рзаев Али Эльдар оглы'.encode('utf-8')[:8]
key = 'Результат разбивается на восемь'.encode('utf-8')[:32]

print('2 rounds')
print('plaintext:\t{}'.format(plaintext.hex()))
ciphertext = encrypt(plaintext, key, 2)
print('ciphertext:\t{}'.format(ciphertext.hex()))
plaintext = decrypt(ciphertext, key, 2)
print('plaintext:\t{}'.format(plaintext.hex()))

print('32 rounds')
print('plaintext:\t{}'.format(plaintext.hex()))
ciphertext = encrypt(plaintext, key)
print('ciphertext:\t{}'.format(ciphertext.hex()))
plaintext = decrypt(ciphertext, key)
print('plaintext:\t{}'.format(plaintext.hex()))

2 rounds
plaintext:	d0a0d0b7d0b0d0b5
ciphertext:	d7f67de36ecf9d41
plaintext:	d0a0d0b7d0b0d0b5
32 rounds
plaintext:	d0a0d0b7d0b0d0b5
ciphertext:	20619a2d78b7c8a5
plaintext:	d0a0d0b7d0b0d0b5


In [76]:
# Задание #3

import math
import random

def isprime(num):
	if num <= 1:
		return False

	for i in range(2, math.ceil(math.sqrt(num))):
		if num % i == 0:
			return False

	return True


def newkeys(p, q):
    n = p * q
    f = (p - 1) * (q - 1)

    E = [n for n in range(f) if isprime(n) and f % n != 0]
    e = random.choice(E)

    d = 1
    while (d * e) % f != 1:
        d += 1
    
    return (e, n), (d, n)


def encrypt(message, pubkey):
    e, n = pubkey
    return list(pow(ord(c), e, n) for c in message)


def decrypt(message, privkey):
    d, n = privkey
    return ''.join(chr(pow(c, d, n)) for c in message)


random.seed(0)
pubkey, privkey = newkeys(277, 521)
plaintext = 'Рзаев Али Эльдар оглы'
print('plaintext:\t{}'.format(plaintext))
ciphertext = encrypt(plaintext, pubkey)
print('ciphertext:\t{}'.format(ciphertext))
plaintext = decrypt(ciphertext, privkey)
print('plaintext:\t{}'.format(plaintext))

plaintext:	Рзаев Али Эльдар оглы
ciphertext:	[28829, 26659, 76777, 140492, 22348, 110918, 13346, 17636, 72212, 110918, 83109, 17636, 109087, 24258, 76777, 63132, 110918, 67467, 121854, 17636, 59341]
plaintext:	Рзаев Али Эльдар оглы


In [94]:
# Задание #4

def hash_(message):
    p = 277
    q = 521
    
    hashvalue = 0
    for i, c in enumerate(message):
        hashvalue = (hashvalue + pow(p * q, i, 1 << 128) * ord(c) + 1) & ~(1 << 128)
    return hashvalue.to_bytes(16, 'little')


plaintext = 'Рзаев'
print('plaintext:\t{}'.format(plaintext))
hashvalue = hash_(plaintext)
print('hash:\t{}'.format(hashvalue.hex()))

plaintext:	Рзаев
hash:	eb8175d907e37297a762000000000000


In [4]:
# Задание #5

import math
import random

def isprime(num):
	if num <= 1:
		return False

	for i in range(2, math.ceil(math.sqrt(num))):
		if num % i == 0:
			return False

	return True


def newkeys(p, q):
    n = p * q
    f = (p - 1) * (q - 1)

    E = [n for n in range(f) if isprime(n) and f % n != 0]
    e = random.choice(E)

    d = 1
    while (d * e) % f != 1:
        d += 1
    
    return (e, n), (d, n)


def encrypt(message, pubkey):
    e, n = pubkey
    return list(pow(ord(c), e, n) for c in message)


def decrypt(message, privkey):
    d, n = privkey
    return ''.join(chr(pow(c, d, n)) for c in message)


def hash_(message):
    p = 277
    q = 521
    
    hashvalue = 0
    for i, c in enumerate(message):
        hashvalue = (hashvalue + pow(p * q, i, 1 << 128) * ord(c) + 1) & ((1 << 128) - 1)
    return hashvalue.to_bytes(16, 'little')


random.seed(0)
pubkey, privkey = newkeys(277, 521)
plaintext = 'Рзаев Али Эльдар оглы'
print('plaintext:\t{}'.format(plaintext))
hashvalue = hash_(plaintext)
print('hash:\t{}'.format(hashvalue.hex()))
signature = encrypt(hashvalue.hex(), privkey)
print('signature:\t{}'.format(signature))
hashvalue = decrypt(signature, pubkey)
print('hash:\t{}'.format(hashvalue))

plaintext:	Рзаев Али Эльдар оглы
hash:	a96401315040bfe9ff6a7b24cf77abf7
signature:	[31077, 61504, 39914, 91406, 111041, 59024, 31032, 59024, 8279, 111041, 91406, 111041, 69264, 37159, 118285, 61504, 37159, 37159, 39914, 31077, 14035, 69264, 97436, 91406, 18454, 37159, 14035, 14035, 31077, 69264, 37159, 14035]
hash:	a96401315040bfe9ff6a7b24cf77abf7
